> **How to run this notebook (command-line)?**
1. Install the `ReinventCommunity` environment:
`conda env create -f environment.yml`
2. Activate the environment:
`conda activate ReinventCommunity`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `REINVENT 3.0`: reinforcement learning selectivity demo
This illustrates the use of a selectivity score component. This component is normally used in a scenario where ligands are designed to bind to a concrete target and at the same time should not be binding to one or more off-targets. The example here looks at a single off-target case.

We will demonstrate how to set up a `REINVENT` run that optimizes molecules to be selective against _Aurora_ kinase by minimizing at the same time their affinity to _B RAF_ kinase.

NOTE: There is a detailed reasoning for each code block provided in the `Reinforcement Learning Demo` notebook.


## 1. Set up the paths
_Please update the following code block such that it reflects your system's installation and execute it._

In [1]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("~/Desktop/reinventcli")
reinvent_env = os.path.expanduser("~/miniconda3/envs/reinvent.v3.0")
output_dir = os.path.expanduser("~/Desktop/REINVENT_RL_Selectivity_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

## 2. Setting up the configuration 
In the cells below we will build a nested dictionary object that will be eventually converted to JSON file which in turn will be consumed by `REINVENT`. 
You can find this file in your `output_dir` location.

### A) Declare the run type

In [2]:
# initialize the dictionary
configuration = {
    "version": 3,                          # we are going to use REINVENT's newest release
    "run_type": "reinforcement_learning"   # other run types: "sampling", "validation",
                                           #                  "transfer_learning",
                                           #                  "scoring" and "create_model"
}

### B) Sort out the logging details
This includes `result_folder` path where the results will be produced.

Also: `REINVENT` can send custom log messages to a remote location. We have retained this capability in the code. if the `recipient` value differs from `"local"` `REINVENT` will attempt to POST the data to the specified `recipient`. 

In [3]:
# add block to specify whether to run locally or not and
# where to store the results and logging
configuration["logging"] = {
    "sender": "http://0.0.0.1",          # only relevant if "recipient" is set to "remote"
    "recipient": "local",                  # either to local logging or use a remote REST-interface
    "logging_frequency": 10,               # log every x-th steps
    "logging_path": os.path.join(output_dir, "progress.log"), # load this folder in tensorboard
    "result_folder": os.path.join(output_dir, "results"),         # will hold the compounds (SMILES) and summaries
    "job_name": "Reinforcement learning demo",                # set an arbitrary job name for identification
    "job_id": "demo"                       # only relevant if "recipient" is set to a specific REST endpoint
}

Create `"parameters"` field

In [4]:
# add the "parameters" block
configuration["parameters"] = {}

### C) Set Diversity Filter
During each step of Reinforcement Learning the compounds scored above `minscore` threshold are kept in memory. Those scored smiles are written out to a file in the results folder `scaffold_memory.csv`.

In [5]:
# add a "diversity_filter"
configuration["parameters"]["diversity_filter"] =  {
    "name": "IdenticalMurckoScaffold",     # other options are: "IdenticalTopologicalScaffold", 
                                           #                    "NoFilter" and "ScaffoldSimilarity"
                                           # -> use "NoFilter" to disable this feature
    "nbmax": 25,                           # the bin size; penalization will start once this is exceeded
    "minscore": 0.4,                       # the minimum total score to be considered for binning
    "minsimilarity": 0.4                   # the minimum similarity to be placed into the same bin
}

### D) Set Inception
* `smiles` provide here a list of smiles to be incepted 
* `memory_size` the number of smiles allowed in the inception memory
* `sample_size` the number of smiles that can be sampled at each reinforcement learning step from inception memory

In [6]:
# prepare the inception (we do not use it in this example, so "smiles" is an empty list)
configuration["parameters"]["inception"] = {
    "smiles": [],                          # fill in a list of SMILES here that can be used (or leave empty)
    "memory_size": 100,                    # sets how many molecules are to be remembered
    "sample_size": 10                      # how many are to be sampled each epoch from the memory
}

### E) Set the general Reinforcement Learning parameters
* `n_steps` is the amount of Reinforcement Learning steps to perform. Best start with 1000 steps and see if thats enough.
* `agent` is the generative model that undergoes transformation during the Reinforcement Learning run.

We reccomend keeping the other parameters to their default values.

In [7]:
# set all "reinforcement learning"-specific run parameters
configuration["parameters"]["reinforcement_learning"] = {
    "prior": os.path.join(ipynb_path, "models/random.prior.new"), # path to the pre-trained model
    "agent": os.path.join(ipynb_path, "models/random.prior.new"), # path to the pre-trained model
    "n_steps": 1000,                       # the number of epochs (steps) to be performed; often 1000
    "sigma": 128,                          # used to calculate the "augmented likelihood", see publication
    "learning_rate": 0.0001,               # sets how strongly the agent is influenced by each epoch
    "batch_size": 128,                     # specifies how many molecules are generated per epoch
    "reset": 0,                            # if not '0', the reset the agent if threshold reached to get
                                           # more diverse solutions
    "reset_score_cutoff": 0.5,             # if resetting is enabled, this is the threshold
    "margin_threshold": 50                 # specify the (positive) margin between agent and prior
}

### F) Define the scoring function
We will use a `custom_product` type. The component types included are:
* `predictive_property` which is the target activity to _Aurora_ kinase represented by the predictive `regression` model. Note that we set the weight of this component to be the highest.
* `matching_substructure` the `"smiles"` field actually can take both list of SMILES or SMARTS. Normally we reccomend using one or a couple of SMARTS patterns defining your pharmacophore.
* `custom_alerts` the `"smiles"` field  also can work with SMILES or SMARTS
* `selectivity` this is the component that maximizes the `delta` between _Aurora_ and _B RAF_ kinases. Note that `delta` is undergoing transformation. Essentially if the individual prediction is higher for _Aurora_ than _B RAF_ the score will be above 0. The `weight` for this component is set lower than `predictive_property`. The relevant fields that might need to be adjusted to your needs are `activity_model_path` and `offtarget_specific_parameters`. 

Note: All models used in this example are regression models


In [8]:
# prepare the scoring function definition and add at the end
scoring_function = {
    "name": "custom_product",              # this is our default one (alternative: "custom_sum")
    "parallel": False,                     # sets whether components are to be executed
                                           # in parallel; note, that python uses "False" / "True"
                                           # but the JSON "false" / "true"

    # the "parameters" list holds the individual components
    "parameters": [

    # add component: an activity model
    {
        "component_type": "predictive_property", # this is a scikit-learn model, returning
                                                 # activity values
        "name": "Aurora kinase",                 # arbitrary name for the component
        "weight": 6,                             # the weight ("importance") of the component (default: 1)                       # list of SMILES (not required for this component)
        "specific_parameters": {
            "model_path": os.path.join(ipynb_path, "models/Aurora_model.pkl"),   # absolute model path 
            "transformation": {
                "transformation_type": "sigmoid",  # see description above
                "high": 9,                         # parameter for sigmoid transformation
                "low": 4,                          # parameter for sigmoid transformation
                "k": 0.25                          # parameter for sigmoid transformation
                },
            "scikit": "regression",                # model can be "regression" or "classification"
            "descriptor_type": "ecfp_counts",      # sets the input descriptor for this model
            "size": 2048,                          # parameter of descriptor type
            "radius": 3,                           # parameter of descriptor type
            "use_counts": True,                    # parameter of descriptor type
            "use_features": True                   # parameter of descriptor type
        }
    },

    # add component: enforce the match to a given substructure
    {
        "component_type": "matching_substructure", 
        "name": "Matching substructure",       # arbitrary name for the component
        "weight": 1,                           # the weight of the component (default: 1)
        "specific_parameters": {
            "smiles": ["c1ccccc1CC"]           # a match with this substructure is required
        }
    },

    # add component: enforce to NOT match a given substructure
    {
        "component_type": "custom_alerts",
        "name": "Custom alerts",               # arbitrary name for the component
        "weight": 1,                           # the weight of the component (default: 1)
        "specific_parameters": {
            "smiles": [                        # specify the substructures (as list) to penalize
            "[*;r8]",
            "[*;r9]",
            "[*;r10]",
            "[*;r11]",
            "[*;r12]",
            "[*;r13]",
            "[*;r14]",
            "[*;r15]",
            "[*;r16]",
            "[*;r17]",
            "[#8][#8]",
            "[#6;+]",
            "[#16][#16]",
            "[#7;!n][S;!$(S(=O)=O)]",
            "[#7;!n][#7;!n]",
            "C#C",
            "C(=[O,S])[O,S]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#16;!s]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#7;!n]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#8;!o]",
            "[#8;!o][C;!$(C(=[O,N])[N,O])][#16;!s]",
            "[#8;!o][C;!$(C(=[O,N])[N,O])][#8;!o]",
            "[#16;!s][C;!$(C(=[O,N])[N,O])][#16;!s]"
            ]
        }
    },

    # add component: selectivity
    {
      "component_type": "selectivity",
      "name": "B-RAF selectivity",
      "weight": 3,
      "specific_parameters": {
        "activity_specific_parameters": {
            "model_path": os.path.join(ipynb_path, "models/Aurora_model.pkl"),
            "scikit": "regression",
            "descriptor_type": "ecfp_counts",
            "size": 2048,
            "radius": 3,
            "use_counts": True,
            "use_features": True
            },
        "offtarget_specific_parameters": {
            "model_path": os.path.join(ipynb_path, "models/B-RAF_model.pkl"),
            "scikit": "regression",
            "descriptor_type": "ecfp_counts",
            "size": 2048,
            "radius": 3,
            "use_counts": True,
            "use_features": True
            },
        "delta_transformation_parameters": {
            "transformation_type": "sigmoid",
            "high": 3,
            "k": 0.25,
            "low": 0
            }
        }
    }
]}
configuration["parameters"]["scoring_function"] = scoring_function

#### NOTE:  Getting the selectivity score component to reach satisfactory levels is non-trivial and might take considerably higher number of steps

## 3. Write out the configuration

We now have successfully filled the dictionary and will write it out as a `JSON` file in the output directory. Please have a look at the file before proceeding in order to see how the paths have been inserted where required and the `dict` -> `JSON` translations (e.g. `True` to `true`) have taken place.

In [9]:
# write the configuration file to the disc
configuration_JSON_path = os.path.join(output_dir, "RL_config.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

## 4. Run `REINVENT`
Now it is time to execute `REINVENT` locally. Note, that depending on the number of epochs (steps) and the execution time of the scoring function components, this might take a while. As we have only specified a low number of epochs (125) and all components should be fairly quick, this should not take too long in our case though.

The command-line execution looks like this:
```
# activate envionment
conda activate reinvent.v3.0

# execute REINVENT
python <your_path>/input.py <config>.json
```

In [10]:
%%capture captured_err_stream --no-stderr

# execute REINVENT from the command-line
!{reinvent_env}/bin/python {reinvent_dir}/input.py {configuration_JSON_path}

In [11]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

# prepare the output to be parsed
list_epochs = re.findall(r'INFO.*?local', captured_err_stream.stdout, re.DOTALL)
data = [epoch for idx, epoch in enumerate(list_epochs) if idx in [1, 75, 124]]
data = ["\n".join(element.splitlines()[:-1]) for element in data]

We have calculated a total of 125 epochs, let us quickly investigate how the agent fared. Below you see the print-out of the first, one from the middle and the last epoch, respectively. Note, that the fraction of valid `SMILES` is high right from the start (because we use a pre-trained prior). You can see the partial scores for each component for the first couple of compounds, but the most important information is the average score. You can clearly see how it increases over time.

In [12]:
for element in data:
    print(element)

INFO     
 Step 0   Fraction valid SMILES: 97.7   Score: 0.0528   Time elapsed: 0   Time left: 0.0
  Agent     Prior     Target     Score     SMILES
-33.05    -33.05    -22.22      0.08      c1(C)cccc(Sc2cc3c(cc2)N2CCC(=O)C(C)=C2CN3CC2CC2)c1
-33.90    -33.90    -19.17      0.12      C(C)C(=O)N1CCN(C(=O)C(=O)c2c3c(OC)cnc(-n4cc(CC(C)C)nn4)c3[nH]c2)CC1
-26.92    -26.92    -18.03      0.07      C(c1cccc(OC)c1)N1C(=O)C2(CC1)CCN(Cc1cc(F)ccc1)CC2
-23.10    -23.10    -16.20      0.05      C(CC)N(CCC)C(Cc1c(-c2ccc(Cl)cc2)nc2ccccn12)=O
-23.24    -23.24    -16.26      0.05      c1c(Cn2nc3c(cc2=O)CCNC3)ccc(F)c1
-25.86    -25.86    -12.20      0.11      C(C(=O)Nc1ccccc1CC)=C=C
-38.26    -38.26    -38.26      0.00      C(O)(c1cc(C2CC2)c2c(c1)CC(CO)Cn1c-2cc2c1ccc(Cl)c2)=O
-25.34    -25.34    -25.34      0.00      c1cc(S(=O)(NCC(c2cc3c(cc2)OCO3)N2CCCCCC2)=O)ccc1C(=O)C
-23.17    -23.17     -6.10      0.13      c1c(CN2C(=O)C(C)(c3ccc(F)cc3)NC2=O)ccc(F)c1
-23.91    -23.91    -13.93      0.08      c1ccccc

## 5. Analyse the results
In order to analyze the run in a more intuitive way, we can use `tensorboard`:

```
# go to the root folder of the output
cd <your_path>/REINVENT_RL_demo

# make sure, you have activated the proper environment
conda activate reinvent.v3.0

# start tensorboard
tensorboard --logdir progress.log
```

Then copy the link provided to a browser window, e.g. "http://workstation.url.com:6006/". The following figures are exmaple plots - remember, that there is always some randomness involved. In `tensorboard` you can monitor the individual scoring function components. What you see is, that all of those depicted went up (and `Fraction_valid_SMILES` was high troughout). Not shown is the predictive model, which did not perform all that well, so you might want to consider a higher weight next time.

![](img/individual_components.png)

Also the total score increased over time.

![](img/total_score.png)

It might also be informative to look at the results from the prior (dark blue), the agent (blue) and the augmented likelihood (purple) over time.

![](img/likelihood.png)

And last but not least, there is a "Images" tab available that lets you browse through the compounds generated in an easy way. In the molecules, the substructure matches that were defined to be required are highlighted in red (if present). Also, the total scores are given per molecule.

![](img/molecules.png)

The results folder will hold four different files: the agent (pickled), the input JSON (just for reference purposes), the memory (highest scoring compounds in `CSV` format) and the scaffold memory (in `CSV` format).

In [13]:
!head -n 15 {output_dir}/results/memory.csv

,smiles,score,likelihood
49,c1cc(C2(C(F)(F)F)NC(=O)C(C(=O)NS(Cc3ccccc3)(=O)=O)=C(c3cn(CC4CCOCC4)c(=O)cc3)C2)c(F)cc1-c1cncnc1N1C2CC(C(c3cocc3)n3nncc3)CC2CC1,0.9174115,-117.29756
122,c1c(C2(C(F)(F)F)NC(=O)C(C(=O)NS(C3CC3)(=O)=O)=C(c3cn(CC4CCOCC4)c(=O)cc3)C2)c(F)ccc1-c1cncnc1N1C2CC(C(c3ccc(C(F)F)o3)C)CC2CC1,0.9160371,-107.46117
50,c1c(C2(C(F)(F)F)NC(=O)C(C(=O)NS(Cc3ccccc3)(=O)=O)=C(c3cn(CC4CCOCC4)c(=O)cc3)C2)c(F)ccc1-c1cncnc1N1C2CC(C(c3ccc(C(F)F)o3)(C)C)CC2CC1,0.9159551,-106.201866
51,c1(C2(C(F)(F)F)NC(=O)C(C(=O)NS(Cc3ccncc3)(=O)=O)=C(c3cn(CC4CCOCC4)c(=O)cc3)C2)c(F)ccc(-c2cncnc2N2C3CC(CSc4sccc4)CC3CC2)c1,0.91587687,-103.7542
36,c1(C2(C(F)(F)F)NC(=O)C(C(=O)NS(Cc3ccncc3)(=O)=O)=C(c3cn(CC4CCOCC4)c(=O)cc3)C2)c(F)ccc(-c2cncnc2N2C3CC(CSc4sccn4)CC3CC2)c1,0.91587687,-100.189095
116,c1c(C2(C(F)(F)F)NC(=O)C(C(=O)NS(Cc3ccccc3)(=O)=O)=C(c3cn(CC4CCOCC4)c(=O)cc3)C2)c(F)ccc1-c1cncnc1N1C2CC(C(c3ccncc3F)F)CC2CC1,0.9143506,-112.0386
29,c1c(C2(C(F)(F)F)NC(=O)C(C(=O)NS(Cc3ccncc3)(=O)=O)=C(c3cn(CC4CCOCC4)c(=O